In [1]:
import os 
from dotenv import load_dotenv
import json

from pathlib import Path

from mistralai import Mistral


import json
import os
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Set

from dotenv import load_dotenv

# Add the project root to Python path for imports
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from bill_parser_engine.core.reference_resolver.bill_splitter import BillSplitter
from bill_parser_engine.core.reference_resolver.target_identifier import TargetArticleIdentifier
from bill_parser_engine.core.reference_resolver.models import BillChunk, TargetArticle
from bill_parser_engine.core.reference_resolver.config import MISTRAL_MODEL
from bill_parser_engine.core.reference_resolver.original_text_retriever import OriginalTextRetriever

from bill_parser_engine.core.reference_resolver.pipeline import BillProcessingPipeline

from pylegifrance import recherche_code
from pylegifrance.models.constants import CodeNom



load_dotenv('../.env.local')

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key)

print(f'Using mistral model {MISTRAL_MODEL}')

Using mistral model magistral-medium-2506


In [2]:
# Define file paths
bill_file = project_root / "data" / "legal_bill" / "sampled_legislative_bill.md"
output_dir = project_root / "scripts " / "output"
log_file_path = project_root / "notebooks" / f"detailed_reconstruction_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"


In [3]:
pipeline = BillProcessingPipeline(use_cache=True, log_file_path=log_file_path)

2025-06-21 10:42:27,176 - bill_parser_engine.core.reference_resolver.instruction_decomposer - INFO - InstructionDecomposer initialized with caching: enabled
2025-06-21 10:42:27,190 - bill_parser_engine.core.reference_resolver.operation_applier - INFO - OperationApplier initialized with caching: enabled
2025-06-21 10:42:27,204 - bill_parser_engine.core.reference_resolver.result_validator - INFO - ResultValidator initialized with caching: enabled
2025-06-21 10:42:27,205 - bill_parser_engine.core.reference_resolver.legal_amendment_reconstructor - INFO - LegalAmendmentReconstructor initialized with caching: enabled, log file: /Users/duphan/Projects/bill-parser-engine/notebooks/detailed_reconstruction_log_20250621_104227.txt
2025-06-21 10:42:27,206 - bill_parser_engine.core.reference_resolver.pipeline - INFO - BillProcessingPipeline initialized with LegalAmendmentReconstructor (component-level caching enabled)
2025-06-21 10:42:27,207 - bill_parser_engine.core.reference_resolver.pipeline - I

In [4]:
pipeline.load_legislative_text_from_file(bill_file)

2025-06-21 10:42:27,210 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Loaded legislative text: 8696 characters
2025-06-21 10:42:27,212 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Loaded legislative text from file: /Users/duphan/Projects/bill-parser-engine/data/legal_bill/sampled_legislative_bill.md


In [5]:
pipeline.clear_component_cache('target_identifier')
pipeline.clear_component_cache("original_text_retriever")
pipeline.clear_component_cache("text_reconstructor")
#pipeline.clear_component_cache("reference_locator")
#pipeline.clear_component_cache('reference_object_linker')

2025-06-21 10:42:27,221 - bill_parser_engine.core.reference_resolver.cache_manager - INFO - Invalidated 18 cache entries for component 'target_identifier_unified'
2025-06-21 10:42:27,222 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Cleared component cache for: target_identifier
2025-06-21 10:42:27,224 - bill_parser_engine.core.reference_resolver.cache_manager - INFO - Invalidated 10 cache entries for component 'original_text_retriever'
2025-06-21 10:42:27,225 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Cleared component cache for: original_text_retriever
2025-06-21 10:42:27,228 - bill_parser_engine.core.reference_resolver.cache_manager - INFO - Invalidated 17 cache entries for component 'instruction_decomposer'
2025-06-21 10:42:27,233 - bill_parser_engine.core.reference_resolver.cache_manager - INFO - Invalidated 29 cache entries for component 'operation_applier'
2025-06-21 10:42:27,235 - bill_parser_engine.core.reference_resolver.cache_manager - IN

In [6]:
chunks = pipeline.step_1_split_bill()

2025-06-21 10:42:27,239 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 1: Splitting legislative bill into chunks...
2025-06-21 10:42:27,242 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Split into 21 chunks


In [7]:
chunks[4]

BillChunk(text="a) Le I est ainsi modifié : à la fin du 1°, les mots : « mentionnée aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 du 21 octobre 2009 » ; au 2°, les mots : « mentionnée aux 1° ou 2° du II ou au IV de l'article L. 254-1 » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » et, à la fin, les mots : « de ce II » sont remplacés par les mots : « du II de l'article L. 254-1 » ; au 3°, les mots : « mentionnée, d'une part, au 3° du II de l'article L. 254-1 et, d'autre part, aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « , d'une part, mentionnée au 3° du II de l'article L. 254-1 et, d'autre part, de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » ;", titre_text='# TITRE Iᴱᴿ', article_label='Article 1ᵉʳ', article_introductory_phrase='Le code rural et de la 

In [8]:
target_results = pipeline.step_2_identify_target_articles()

2025-06-21 10:42:27,253 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 2: Identifying target articles for 21 chunks...


✓ Detected pure versioning metadata for chunk # TITRE Iᴱᴿ::Article 1ᵉʳ::1°: 1° (Supprimé)
→ Processing chunk with unified LLM-based target identification: # TITRE Iᴱᴿ::Article 1ᵉʳ::2°::a)


2025-06-21 10:42:32,441 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Cached result for future use
→ Processing chunk with unified LLM-based target identification: # TITRE Iᴱᴿ::Article 1ᵉʳ::2°::b)


2025-06-21 10:42:36,433 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


...

✓ Cached result for future use
→ Processing chunk with unified LLM-based target identification: # TITRE Iᴱᴿ::Article 1ᵉʳ::6° ter
⏱️ TargetArticleIdentifier: Waiting 2.8s for rate limiting...


2025-06-21 10:43:33,756 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Cached result for future use
→ Processing chunk with unified LLM-based target identification: # TITRE Iᴱᴿ::Article 1ᵉʳ::7°
⏱️ TargetArticleIdentifier: Waiting 2.4s for rate limiting...


2025-06-21 10:43:37,362 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 10:43:37,369 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Target identification complete: 11 unique articles identified


✓ Cached result for future use


In [9]:
target_results

[{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::2°::a)',
  'chunk_text_preview': 'a) (nouveau) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés ...',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '2°', 'a)'],
  'target_article': {'operation_type': 'MODIFY',
   'code': 'code rural et de la pêche maritime',
   'article': 'L. 254-1',
   'full_citation': 'code rural et de la pêche maritime::L. 254-1',
   'confidence': 1.0,
   'raw_text': 'a) (nouveau) Au 3° du II, les mots : « prévu aux a...'}},
 {'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::2°::b)',
  'chunk_text_preview': 'b) Le VI est ainsi modifié : à la fin de la première phrase, les mots : « incompatible avec celui de...',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '2°', 'b)'],
  'target_article': {'operation_type': 'MODIFY',
   'code': 'code rural et de la pêche maritime',
   'article': 'L. 254-1',
   'full_citation': 'code rural et de la pêche maritime::L. 254-1',
   'confidence': 1.0,
   'raw_

In [10]:
weird_issues = []
for res in target_results:
    if res.get('target_article').get('code') is None: 
        print(res)
        weird_issues.append(res)
        print('----')

In [11]:
len(weird_issues)

0

In [12]:
len(target_results)

18

In [13]:
retrieval_results = pipeline.step_3_retrieve_original_texts()
retrieval_results

2025-06-21 10:43:37,417 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 3: Retrieving original text for target articles...
2025-06-21 10:43:37,418 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - → Fetching article L. 254-1 from code rural et de la pêche maritime
2025-06-21 10:43:37,565 - pylegifrance.auth - INFO - Legifrance API authentication successful.
2025-06-21 10:43:37,567 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/search
2025-06-21 10:43:38,184 - pylegifrance.client - INFO - API call to 'search' successful.
2025-06-21 10:43:38,186 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1
2025-06-21 10:43:38,187 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle
2025-06-21 10:43:38,361 - pylegifrance.client - INFO - API call to 'consult/getArticle' successful.
2025-06-21 10:43

[{'article_key': 'code rural et de la pêche maritime::L. 254-1',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1',
  'operation_type': 'MODIFY',
  'original_text': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant pour leur compte, y compris les groupements d'achats ; 2° L'application, en qualité de prestataire de services, des produits phytopharmaceutiques, sauf si elle est effectuée dans le cadre de contrats d'entraide à titre gratuit au sens de l'article L. 325-1 ou par un exploitant agricole titulaire du certificat mentionné au II de l'article L. 254-3 sur des exploitations dont la surface agricole utile est infé

In [14]:
target_results[5]

{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::3° ter::b)',
 'chunk_text_preview': 'b) Le second alinéa est supprimé ;',
 'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '3° ter', 'b)'],
 'target_article': {'operation_type': 'MODIFY',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1-2',
  'full_citation': 'code rural et de la pêche maritime::L. 254-1-2',
  'confidence': 1.0,
  'raw_text': 'b) Le second alinéa est supprimé ;'}}

In [15]:
pipeline._analyze_retrieval_results(retrieval_results)

{'total_articles': 11,
 'successful_retrievals': 11,
 'failed_retrievals': 0,
 'success_rate': 1.0,
 'source_stats': {'pylegifrance': 10, 'insert_operation': 1},
 'text_length_stats': {'average': 1479.0,
  'maximum': 3321,
  'minimum': 0,
  'total_characters': 16269},
 'failed_articles': []}

In [ ]:
reconstruction_results = pipeline.step_4_reconstruct_texts()


2025-06-21 10:43:45,944 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 4: Applying text reconstruction...
2025-06-21 10:43:45,945 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Processing 18 chunks with identified target articles (skipping 3 chunks with pure versioning metadata or errors)
2025-06-21 10:43:45,946 - bill_parser_engine.core.reference_resolver.legal_amendment_reconstructor - INFO - Starting amendment reconstruction for article code rural et de la pêche maritime::L. 254-1 - instruction: a) (nouveau) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés ...
2025-06-21 10:43:45,947 - bill_parser_engine.core.reference_resolver.instruction_decomposer - INFO - Parsing amendment instruction: a) (nouveau) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés par les mots : « à l'utilisation des produits phytopharmaceutiques » ;...
2025-06-21 10:43:49,314 - httpx - INFO - HTTP Req

⏱️ OperationApplier: Waiting 0.1s for rate limiting...


2025-06-21 10:44:11,402 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 10:44:11,406 - bill_parser_engine.core.reference_resolver.operation_applier - INFO - Operation applied - Success: True (processing time: 22083ms)
2025-06-21 10:44:11,408 - bill_parser_engine.core.reference_resolver.result_validator - INFO - Validating legal coherence for 1 operations


In [ ]:
pipeline._analyze_reconstruction_results(reconstruction_results)#.get('failed_chunks')

In [ ]:
#pipeline.export_traces_after_step('step_4')

In [18]:
failed_reconstruction = []
for res in reconstruction_results:
    if res.get('chunk_id') in pipeline._analyze_reconstruction_results(reconstruction_results).get('failed_chunks'):
        failed_reconstruction.append(res)

In [19]:
failed_reconstruction

[{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::1°',
  'chunk_text_preview': '1° (Supprimé)',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '1°'],
  'target_article': None,
  'reconstruction_result': None,
  'error': 'No target article identified'},
 {'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::3°',
  'chunk_text_preview': '3° (Supprimé)',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '3°'],
  'target_article': None,
  'reconstruction_result': None,
  'error': 'No target article identified'},
 {'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::5°',
  'chunk_text_preview': '5° (Supprimé)',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '5°'],
  'target_article': None,
  'reconstruction_result': None,
  'error': 'No target article identified'},
 {'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::6°::a)',
  'chunk_text_preview': "a) (nouveau) Au premier alinéa, les mots : « , et notamment la désignation de l'autorité administrat...",
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '6°', 'a)'],
  'target_a

In [20]:
len(failed_reconstruction)

42

In [ ]:
located_ref = pipeline.step_5_locate_references()

In [ ]:
pipeline._analyze_reference_location_results(located_ref)

In [ ]:
located_ref[2]

In [ ]:
ref_link = pipeline.step_6_link_references()

In [ ]:
pipeline._analyze_reference_linking_results(ref_link)